In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import OneHotEncoder

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:

url = '/content/drive/MyDrive/Data Mining/movies.csv'
data = pd.read_csv(url)

data.head()

,name,rating,genre,year,released,score,votes,director,writer,star,country,budget,gross,company,runtime
0,The Shining,R,Drama,1980,"June 13, 1980 (United States)",8.4,927000.0,Stanley Kubrick,Stephen King,Jack Nicholson,United Kingdom,19000000.0,46998772.0,Warner Bros.,146.0
1,The Blue Lagoon,R,Adventure,1980,"July 2, 1980 (United States)",5.8,65000.0,Randal Kleiser,Henry De Vere Stacpoole,Brooke Shields,United States,4500000.0,58853106.0,Columbia Pictures,104.0
2,Star Wars: Episode V - The Empire Strikes Back,PG,Action,1980,"June 20, 1980 (United States)",8.7,1200000.0,Irvin Kershner,Leigh Brackett,Mark Hamill,United States,18000000.0,538375067.0,Lucasfilm,124.0
3,Airplane!,PG,Comedy,1980,"July 2, 1980 (United States)",7.7,221000.0,Jim Abrahams,Jim Abrahams,Robert Hays,United States,3500000.0,83453539.0,Paramount Pictures,88.0
4,Caddyshack,R,Comedy,1980,"July 25, 1980 (United States)",7.3,108000.0,Harold Ramis,Brian Doyle-Murray,Chevy Chase,United States,6000000.0,39846344.0,Orion Pictures,98.0


In [4]:

if isinstance(data['genre'].iloc[0], str):
  data['genre'] = data['genre'].apply(lambda x: x.split(','))
else:
  print("Genre column already contains lists. Skipping split.")
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
genre_matrix = encoder.fit_transform(data['genre'].explode().reset_index(drop=True).to_frame())
genre_df = pd.DataFrame(genre_matrix, columns=encoder.get_feature_names_out(['genre']))
data = pd.concat([data.reset_index(drop=True), genre_df], axis=1)

In [5]:
movie_features = genre_df
movie_similarity = cosine_similarity(movie_features)
movie_similarity_df = pd.DataFrame(movie_similarity, index=data['name'], columns=data['name'])

In [6]:
def get_similar_movies(movie_name, num_recommendations=5):
    if movie_name not in movie_similarity_df.index:
        return f"Movie '{movie_name}' not found in the dataset."

    similar_scores = movie_similarity_df[movie_name].sort_values(ascending=False)
    similar_movies = similar_scores.iloc[1:num_recommendations + 1]
    return similar_movies

In [7]:
# Example: Find movies similar to "The Shining"

similar_movies = get_similar_movies("The Shining")
print(similar_movies)

name
54                 1.0
The Celebration    1.0
Scandal            1.0
Valmont            1.0
Lean on Me         1.0
Name: The Shining, dtype: float64


In [8]:
# Displaying the recommended movies with their ratings

recommended_movies = data[data['name'].isin(similar_movies.index)][['name', 'rating']]
recommended_movies

,name,rating
1508,Lean on Me,PG-13
1513,Valmont,R
1514,Scandal,R
3285,The Celebration,R
3302,54,R
